In [1]:
import pandas as pd
import numpy as np

np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow.keras import layers

In [2]:
df_train = pd.read_csv('Data/train.csv')
print(df_train.shape)
df_train.head()

(42000, 785)


,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df_test = pd.read_csv('Data/test.csv')
print(df_test.shape)
df_test.head()

(28000, 784)


,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
from sklearn.model_selection import train_test_split

In [5]:
x = df_train.drop(columns=['label'], axis=1)
y = df_train.label

test = df_test

In [6]:
x = x / 255.0

test = test / 255.0

In [7]:
x = x.values.reshape(-1, 28, 28, 1)

test = test.values.reshape(-1, 28, 28, 1)

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])

In [10]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [11]:
model.fit(x_train, y_train, epochs=10)

Epoch 1/10
919/919 [==============================] - 2s 2ms/step - loss: 0.3928 - accuracy: 0.8869
Epoch 2/10
919/919 [==============================] - 2s 2ms/step - loss: 0.1893 - accuracy: 0.9429
Epoch 3/10
919/919 [==============================] - 2s 2ms/step - loss: 0.1409 - accuracy: 0.9580
Epoch 4/10
919/919 [==============================] - 2s 2ms/step - loss: 0.1111 - accuracy: 0.9670
Epoch 5/10
919/919 [==============================] - 2s 2ms/step - loss: 0.0926 - accuracy: 0.9721
Epoch 6/10
919/919 [==============================] - 2s 2ms/step - loss: 0.0780 - accuracy: 0.9759
Epoch 7/10
919/919 [==============================] - 2s 2ms/step - loss: 0.0706 - accuracy: 0.9769
Epoch 8/10
919/919 [==============================] - 1s 2ms/step - loss: 0.0602 - accuracy: 0.9804
Epoch 9/10
919/919 [==============================] - 1s 2ms/step - loss: 0.0524 - accuracy: 0.9821
Epoch 10/10
919/919 [==============================] - 1s 2ms/step - loss: 0.0504 - accuracy: 0.9836

In [12]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)

print('\nTest accuracy:', test_acc)

394/394 - 0s - loss: 0.0952 - accuracy: 0.9720 - 408ms/epoch - 1ms/step

Test accuracy: 0.9719841480255127


In [13]:
probability_model = tf.keras.Sequential([model,
                                         tf.keras.layers.Softmax()])

In [14]:
predictions = probability_model.predict(x_test)

394/394 [==============================] - 0s 743us/step


In [15]:
predictions_single = probability_model.predict(test)

print(predictions_single)

875/875 [==============================] - 1s 804us/step
[[0.    0.    1.    ... 0.    0.    0.   ]
 [1.    0.    0.    ... 0.    0.    0.   ]
 [0.    0.    0.    ... 0.001 0.002 0.995]
 ...
 [0.    0.    0.    ... 0.    0.    0.   ]
 [0.    0.    0.    ... 0.    0.    0.937]
 [0.    0.    1.    ... 0.    0.    0.   ]]


In [16]:
predictions_single = np.argmax(predictions_single, axis=1)

predictions_single = pd.Series(predictions_single, name="Label")
submission = pd.concat([pd.Series(range(1, 28001), name='ImageId'), predictions_single], axis=1)

submission.to_csv("submission.csv", index=False)